# Install Libraries

In [1]:
!pip install folium geopandas
!pip install opencv-python
!pip install html2image

# Import Libraries

In [2]:
import pandas as pd
import geopandas as gpd
import json
from html2image import Html2Image
from pathlib import Path
import numpy as np
import math
import folium
import cv2
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from folium.plugins import HeatMap

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Read CSV Files

Read the CSV file with data about the capacities in public transportation vehicles

In [4]:
gefaessgroesse_df = pd.read_csv('../data/gefaessgroesse.csv', delimiter=';',encoding = "utf-8")
gefaessgroesse_df.head()

,Plan_Fahrt_Id,SITZPLAETZE,KAP_1m2,KAP_2m2,KAP_3m2,KAP_4m2
0,40287,13,15.5,18.0,21.0,23.0
1,60225,13,15.5,18.0,21.0,23.0
2,60178,13,15.5,18.0,21.0,23.0
3,60218,13,15.5,18.0,21.0,23.0
4,60175,13,15.5,18.0,21.0,23.0


Read the CSV file with data about the stops (names of stops, etc.)

In [5]:
haltestellen_df = pd.read_csv('../data/haltestellen.csv', delimiter=';',encoding = "utf-8")
haltestellen_df.head()

,Haltestellen_Id,Haltestellennummer,Haltestellenkurzname,Haltestellenlangname
0,119,1179,HERZ,"Zürich, Herzogenmühlestrasse"
1,104,1186,HEUB,"Zürich, Heubeeriweg"
2,176,1187,HEUR,"Zürich, Heuried"
3,386,6250,HIMM,"Zürich, Himmeri"
4,416,1194,HINT,"Zürich, Hinterbergstrasse"


Read the CSV file with data about the different routes

In [6]:
linie_df = pd.read_csv('../data/linie.csv', delimiter=';',encoding = "utf-8")
linie_df.head()

,Linien_Id,Linienname,VSYS,Linienname_Fahrgastauskunft
0,51,10,T,10
1,36,11,T,11
2,52,12,T,12
3,13,13,T,13
4,14,14,T,14


Read the CSV file with data about the type of day

In [7]:
tagtyp_df = pd.read_csv('../data/tagtyp.csv', delimiter=';',encoding = "utf-8")
tagtyp_df.head()

,Tagtyp_Id,Tagtypname,Bemerkung
0,2,Unbenutzt,NaN
1,7,14-A-23,NaN
2,17,14-B-23,NaN
3,20,14-C-23,NaN
4,32,14-F-23,NaN


Read the main CSV file with data about the passengers and all the links to previous read files

In [8]:
reisende_df = pd.read_csv('../data/reisende.csv', delimiter=';',encoding = "utf-8")
reisende_df.head()

,Tagtyp_Id,Linien_Id,Linienname,Plan_Fahrt_Id,Richtung,Sequenz,Haltestellen_Id,Nach_Hst_Id,FZ_AB,Anzahl_Messungen,...,Besetzung,Distanz,Tage_DTV,Tage_DWV,Tage_SA,Tage_SO,Nachtnetz,Tage_SA_N,Tage_SO_N,ID_Abschnitt
0,21,113,15,116279,1,1,161,160.0,09:52:18,9,...,0.44444,289,33.0,0.0,0,33.0,0,0.0,0.0,16100160
1,21,113,15,116279,1,2,160,159.0,09:53:24,9,...,0.88889,535,33.0,0.0,0,33.0,0,0.0,0.0,16000159
2,21,113,15,116279,1,3,159,158.0,09:54:36,9,...,4.77778,282,33.0,0.0,0,33.0,0,0.0,0.0,15900158
3,21,113,15,116279,1,4,158,26.0,09:55:30,9,...,5.88889,424,33.0,0.0,0,33.0,0,0.0,0.0,15800026
4,21,113,15,116279,1,5,26,25.0,09:57:00,9,...,4.22222,258,33.0,0.0,0,33.0,0,0.0,0.0,2600025


# Additional Datasets

To build a geospatial map, we need additional information about the location of the stops. We found data by VBZ (see https://github.com/VerkehrsbetriebeZuerich/vbz-flow-concept/blob/master/data-treatment-jupyter/vbz-jupyter.ipynb).

Unfortunately, this dataset uses different IDs and we cannot merge it using the `Haltestellen_Id`. However, we can map it to the `haltestellen.csv` using the column `Haltestellenlangname`.

In [9]:
stops = pd.read_csv('../data/stops.csv')
stops.head()

,GPS_Latitude,GPS_Longitude,Haltestellen_Id,Haltestellenlangname
0,47.452271,8.571438,595,"Zürich Flughafen, Fracht"
1,47.450239,8.563887,594,"Zürich Flughafen, Bahnhof"
2,47.294990,8.564286,749,"Thalwil, Zentrum"
3,47.370167,8.513776,46,"Zürich, Goldbrunnenplatz"
4,47.437911,8.562140,592,"Glattbrugg, Unterriet"


# Merge Tables

After reading the tables, we merge them in one big table for simpler processing and data analysis.

In [10]:
# We merge tables into the main table based on the provided ID
data = reisende_df.merge(gefaessgroesse_df, left_on='Plan_Fahrt_Id', right_on='Plan_Fahrt_Id')
data = data.merge(haltestellen_df, left_on='Haltestellen_Id', right_on='Haltestellen_Id')
data = data.merge(linie_df, left_on='Linien_Id', right_on='Linien_Id')
data = data.merge(tagtyp_df, left_on='Tagtyp_Id', right_on='Tagtyp_Id')

# For the geospatial analysis, we are only interested for buses that are going somewhere (not final stop)
data = data.dropna(subset=['Nach_Hst_Id'])
data['Nach_Hst_Id'] = data['Nach_Hst_Id'].astype(int)

# We merge the haltestellen_df again with a different key so that we obtain information about the
# departing stop and the next stop in the same table
data = data.merge(haltestellen_df, left_on='Nach_Hst_Id', right_on='Haltestellen_Id', suffixes=('_from','_to'))

data.head()

,Tagtyp_Id,Linien_Id,Linienname_x,Plan_Fahrt_Id,Richtung,Sequenz,Haltestellen_Id_from,Nach_Hst_Id,FZ_AB,Anzahl_Messungen,...,Haltestellenlangname_from,Linienname_y,VSYS,Linienname_Fahrgastauskunft,Tagtypname,Bemerkung,Haltestellen_Id_to,Haltestellennummer_to,Haltestellenkurzname_to,Haltestellenlangname_to
0,21,113,15,116279,1,1,161,160,09:52:18,9,...,"Zürich, Milchbuck",15,T,15,77-C-23,NaN,160,1215,HIRS,"Zürich, Hirschwiesenstrasse"
1,21,113,15,116279,1,2,160,159,09:53:24,9,...,"Zürich, Hirschwiesenstrasse",15,T,15,77-C-23,NaN,159,465,BERN,"Zürich, Berninaplatz"
2,21,113,15,116279,1,3,159,158,09:54:36,9,...,"Zürich, Berninaplatz",15,T,15,77-C-23,NaN,158,2151,SALE,"Zürich, Salersteig"
3,21,113,15,116279,1,4,158,26,09:55:30,9,...,"Zürich, Salersteig",15,T,15,77-C-23,NaN,26,2572,SOER,"Zürich, Sternen Oerlikon"
4,21,113,15,116279,1,5,26,25,09:57:00,9,...,"Zürich, Sternen Oerlikon",15,T,15,77-C-23,NaN,25,3034,BOER,"Zürich, Bahnhof Oerlikon"


In [11]:
# Next, we merge the GPS coordinates into the table
data = data.merge(stops, left_on='Haltestellenlangname_from', right_on='Haltestellenlangname', suffixes=('','_from'))
data = data.merge(stops, left_on='Haltestellenlangname_to', right_on='Haltestellenlangname', suffixes=('','_to'))

# remove duplicates that we produced by merging
data = data.loc[:,~data.columns.duplicated()]
data = data.drop(['Haltestellenlangname'], axis=1)

data.head()

,Tagtyp_Id,Linien_Id,Linienname_x,Plan_Fahrt_Id,Richtung,Sequenz,Haltestellen_Id_from,Nach_Hst_Id,FZ_AB,Anzahl_Messungen,...,Bemerkung,Haltestellen_Id_to,Haltestellennummer_to,Haltestellenkurzname_to,Haltestellenlangname_to,GPS_Latitude,GPS_Longitude,Haltestellen_Id,GPS_Latitude_to,GPS_Longitude_to
0,21,113,15,116279,1,1,161,160,09:52:18,9,...,NaN,160,1215,HIRS,"Zürich, Hirschwiesenstrasse",47.397778,8.541750,129,47.400252,8.543470
1,21,113,15,116279,1,2,160,159,09:53:24,9,...,NaN,159,465,BERN,"Zürich, Berninaplatz",47.400252,8.543470,400,47.403543,8.547780
2,21,113,15,116279,1,3,159,158,09:54:36,9,...,NaN,158,2151,SALE,"Zürich, Salersteig",47.403543,8.547780,399,47.406055,8.548362
3,21,113,15,116279,1,4,158,26,09:55:30,9,...,NaN,26,2572,SOER,"Zürich, Sternen Oerlikon",47.406055,8.548362,398,47.410066,8.546233
4,21,113,15,116279,1,5,26,25,09:57:00,9,...,NaN,25,3034,BOER,"Zürich, Bahnhof Oerlikon",47.410066,8.546233,49,47.411274,8.544966


In [12]:
data.columns

Index(['Tagtyp_Id', 'Linien_Id', 'Linienname_x', 'Plan_Fahrt_Id', 'Richtung',
       'Sequenz', 'Haltestellen_Id_from', 'Nach_Hst_Id', 'FZ_AB',
       'Anzahl_Messungen', 'Einsteiger', 'Aussteiger', 'Besetzung', 'Distanz',
       'Tage_DTV', 'Tage_DWV', 'Tage_SA', 'Tage_SO', 'Nachtnetz', 'Tage_SA_N',
       'Tage_SO_N', 'ID_Abschnitt', 'SITZPLAETZE', 'KAP_1m2', 'KAP_2m2',
       'KAP_3m2', 'KAP_4m2', 'Haltestellennummer_from',
       'Haltestellenkurzname_from', 'Haltestellenlangname_from',
       'Linienname_y', 'VSYS', 'Linienname_Fahrgastauskunft', 'Tagtypname',
       'Bemerkung', 'Haltestellen_Id_to', 'Haltestellennummer_to',
       'Haltestellenkurzname_to', 'Haltestellenlangname_to', 'GPS_Latitude',
       'GPS_Longitude', 'Haltestellen_Id', 'GPS_Latitude_to',
       'GPS_Longitude_to'],
      dtype='object')

### Clean Data

The data is already well cleaned. However, we reomve some some connections that we don't want to visualize and round the times to int-values

In [13]:
# For some reason, the time is shifted by 4h
sorted(reisende_df.FZ_AB.str.slice(stop=2).unique())

['04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28']

In [14]:
# therefore, we subtract 4h
data.FZ_AB = data.FZ_AB.str.slice(stop=2).astype(int)
data.FZ_AB %= 24
sorted(data.FZ_AB.unique())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23]

In [15]:
#Delete connections made by cablecars and night liners
data = data[(data.VSYS == 'B') | (data.VSYS == 'TR') | (data.VSYS == 'T') | (data.VSYS == 'N')]

# Calculate some statistics

Next, we calculate the statistics that we want to visualize in our map

In [16]:
# create weekday attribute
data['weekday'] = (data.Tage_SA == 0) & (data.Tage_SO == 0) & (data.Tage_SA_N == 0) & (data.Tage_SO_N == 0)

In [17]:
# Free seats (we clip since Besetzung can be bigger than Sitzplaetze)
data['seat_occupancy'] = data['Besetzung'] / data['SITZPLAETZE']
data['seat_occupancy'] = data['seat_occupancy'].clip(upper=1)
data['free_seats'] = data['SITZPLAETZE'] - data['Besetzung']
data['free_seats'] = data['free_seats'].clip(lower=0)
data.head()

,Tagtyp_Id,Linien_Id,Linienname_x,Plan_Fahrt_Id,Richtung,Sequenz,Haltestellen_Id_from,Nach_Hst_Id,FZ_AB,Anzahl_Messungen,...,Haltestellenkurzname_to,Haltestellenlangname_to,GPS_Latitude,GPS_Longitude,Haltestellen_Id,GPS_Latitude_to,GPS_Longitude_to,weekday,seat_occupancy,free_seats
0,21,113,15,116279,1,1,161,160,9,9,...,HIRS,"Zürich, Hirschwiesenstrasse",47.397778,8.541750,129,47.400252,8.543470,False,0.006734,65.55556
1,21,113,15,116279,1,2,160,159,9,9,...,BERN,"Zürich, Berninaplatz",47.400252,8.543470,400,47.403543,8.547780,False,0.013468,65.11111
2,21,113,15,116279,1,3,159,158,9,9,...,SALE,"Zürich, Salersteig",47.403543,8.547780,399,47.406055,8.548362,False,0.072391,61.22222
3,21,113,15,116279,1,4,158,26,9,9,...,SOER,"Zürich, Sternen Oerlikon",47.406055,8.548362,398,47.410066,8.546233,False,0.089226,60.11111
4,21,113,15,116279,1,5,26,25,9,9,...,BOER,"Zürich, Bahnhof Oerlikon",47.410066,8.546233,49,47.411274,8.544966,False,0.063973,61.77778


# Edges for Geospatial Plot of Connections

### Calculate the values of the edges

First, we aggregate the data we want to visualize

In [18]:
connections = data.groupby(['FZ_AB', 'ID_Abschnitt', 'weekday'])
connections = connections.agg(
    seat_occupancy_mean=('seat_occupancy', 'mean'),
    free_seats_max=('free_seats', 'max'),
    free_seats_mean=('free_seats', 'mean'),
    name_from=('Haltestellenlangname_from', 'first'),
    lat_from=('GPS_Latitude', 'first'),
    long_from=('GPS_Longitude', 'first'),
    name_to=('Haltestellenlangname_to', 'first'),
    lat_to=('GPS_Latitude_to', 'first'),
    long_to=('GPS_Longitude_to', 'first')
)

In [19]:
connections.head()

seat_occupancy_mean  free_seats_max  \
FZ_AB ID_Abschnitt weekday                                        
0     1800467      False               0.011939        90.64286   
                   True                0.010613        90.56410   
      1900466      False               0.035007        90.39130   
                   True                0.027943        90.02564   
      1900467      False               0.039336        89.45455   

                            free_seats_mean         name_from   lat_from  \
FZ_AB ID_Abschnitt weekday                                                 
0     1800467      False          89.913527    Zürich, Rehalp  47.350878   
                   True           90.034204    Zürich, Rehalp  47.350878   
      1900466      False          87.814362  Zürich, Balgrist  47.354413   
                   True           88.457157  Zürich, Balgrist  47.354413   
      1900467      False          87.420469  Zürich, Balgrist  47.354413   

                            long_from                     name_to     lat_to  \
FZ_AB ID_Abschnitt weekday                                                     
0     1800467      False     8.583213  Zürich, Friedhof Enzenbühl  47.351045   
                   True      8.583213  Zürich, Friedhof Enzenbühl  47.351045   
      1900466      False     8.575083            Zürich, Burgwies  47.358070   
                   True      8.575083            Zürich, Burgwies  47.358070   
      1900467      False     8.575083  Zürich, Friedhof Enzenbühl  47.351045   

                             long_to  
FZ_AB ID_Abschnitt weekday            
0     1800467      False    8.580114  
                   True     8.580114  
      1900466      False    8.571704  
                   True     8.571704  
      1900467      False    8.580114

In [20]:
# Remove invalid data (this happens when there are no measurements for some connections)
connections = connections[pd.notna(connections['free_seats_max'])]
connections = connections[pd.notna(connections['free_seats_mean'])]

# Convert to Integer
connections['free_seats_max'] = connections['free_seats_max'].astype(int)
connections['free_seats_mean'] = connections['free_seats_mean'].astype(int)
connections['seat_occupancy_mean'] = connections['seat_occupancy_mean'] * 100
connections['seat_occupancy_mean'] = connections['seat_occupancy_mean'].astype(int)

# Reset index (otherwise, FZ_AB is the index)
connections = connections.reset_index()

# Remove all lines, where start and ziel are identical
connections = connections.query("lat_from != lat_to and long_from != long_to")

connections.head()

,FZ_AB,ID_Abschnitt,weekday,seat_occupancy_mean,free_seats_max,free_seats_mean,name_from,lat_from,long_from,name_to,lat_to,long_to
0,0,1800467,False,1,90,89,"Zürich, Rehalp",47.350878,8.583213,"Zürich, Friedhof Enzenbühl",47.351045,8.580114
1,0,1800467,True,1,90,90,"Zürich, Rehalp",47.350878,8.583213,"Zürich, Friedhof Enzenbühl",47.351045,8.580114
2,0,1900466,False,3,90,87,"Zürich, Balgrist",47.354413,8.575083,"Zürich, Burgwies",47.358070,8.571704
3,0,1900466,True,2,90,88,"Zürich, Balgrist",47.354413,8.575083,"Zürich, Burgwies",47.358070,8.571704
4,0,1900467,False,3,89,87,"Zürich, Balgrist",47.354413,8.575083,"Zürich, Friedhof Enzenbühl",47.351045,8.580114


## Calculate the coordinates of the edges

One issue we face is that trams can travel in both directions. In that case, they have the same coordinates. Therefore, we introduce a helper-function that calculates parallel vectors shifted by some value. This allows to create parallel lines in the visualization so that both directions become visible.

In [21]:
def calculate_shifted_vector(x1, y1, x2, y2, distance):
    # Step 1: Compute the direction of the original vector
    dx = x2 - x1
    dy = y2 - y1
    length = math.sqrt(dx**2 + dy**2)

    # Step 2: Compute the perpendicular direction to the original vector
    # Rotate the original vector by 90 degrees
    perpendicular_dx = -dy
    perpendicular_dy = dx

    # Step 3: Scale the perpendicular direction vector by the desired distance
    scale_factor = distance / length
    shifted_dx = perpendicular_dx * scale_factor
    shifted_dy = perpendicular_dy * scale_factor

    # Step 4: Add the scaled perpendicular vector to both points of the original vector
    new_x1 = x1 + shifted_dx
    new_y1 = y1 + shifted_dy
    new_x2 = x2 + shifted_dx
    new_y2 = y2 + shifted_dy

    return new_x1, new_y1, new_x2, new_y2

# Example usage
x1, y1 = 1, 1
x2, y2 = 3, 3
distance = 0.005

new_x1, new_y1, new_x2, new_y2 = calculate_shifted_vector(x1, y1, x2, y2, distance)
print("New vector coordinates:")
print(f"({new_x1}, {new_y1}) -> ({new_x2}, {new_y2})")

New vector coordinates:
(0.9964644660940672, 1.0035355339059326) -> (2.9964644660940674, 3.0035355339059326)


### Calculate line coordinates

We slightly shorten the lines to make them better visible, shift lines that lie on top of each other, and save them in a dataframe

In [22]:
connections_coord = connections.copy()

 #shorten vector to make it more readable (see comments below)
length_param = 0.0003 # was 0.0005

# we introduce new columns that are required for the if/else switch below
# these columns are deleted later
connections_coord["lat_from_new"] = np.nan
connections_coord["long_from_new"] = np.nan
connections_coord["lat_to_new"] = np.nan
connections_coord["long_to_new"] = np.nan

for index,row in connections_coord.iterrows():

    dlong = row['long_to']-row['long_from']
    dlat = row['lat_to']-row['lat_from']

    # vector that has to be drawn -> does not look good on map
    # vector = (dlat, dlong)

    # We calculate the unit vector and make it a bit smaller than actually needed (stops become better visible)
    length = math.sqrt(dlat**2 + dlong**2)
    unitvector = (dlat/length,dlong/length)

    # Calculate the coordinates of the lines to draw
    lat_from = row['lat_from'] + (length_param * unitvector[0])
    long_from = row['long_from'] + (length_param * unitvector[1])
    lat_to = row['lat_to'] - (length_param * unitvector[0])
    long_to = row['long_to'] - (length_param * unitvector[1])

    # Trams can drive in both directions and will have the same coordinates.
    # If there is already a line on the calculated positions, introduce a small offset.
    if len(connections_coord[(connections_coord.FZ_AB == row.FZ_AB) & (connections_coord.weekday == row.weekday) & (((connections_coord.lat_from_new == lat_from) & (connections_coord.long_from_new == long_from) & (connections_coord.lat_to_new == lat_to) & (connections_coord.long_to_new == long_to)) | ((connections_coord.lat_to_new == lat_from) & (connections_coord.long_to_new == long_from) & (connections_coord.lat_from_new == lat_to) & (connections_coord.long_from_new == long_to))) ]) > 0:
        # Entry already exists -> introduce offset
        lat_from, long_from, lat_to, long_to = calculate_shifted_vector(lat_from, long_from, lat_to, long_to, distance=0.0002)

    connections_coord.loc[index, 'lat_from_new'] = lat_from
    connections_coord.loc[index, 'long_from_new'] = long_from
    connections_coord.loc[index, 'lat_to_new'] = lat_to
    connections_coord.loc[index, 'long_to_new'] = long_to

# Rename columns (delete the temporary ones required for the if-statement above)
connections_coord.drop(['lat_from', 'long_from', 'lat_to', 'long_to'], axis=1, inplace=True)
connections_coord.rename(columns={"lat_from_new": "lat_from", "long_from_new": "long_from", "lat_to_new": "lat_to", "long_to_new": "long_to"}, inplace=True)

connections_coord.head()

,FZ_AB,ID_Abschnitt,weekday,seat_occupancy_mean,free_seats_max,free_seats_mean,name_from,name_to,lat_from,long_from,lat_to,long_to
0,0,1800467,False,1,90,89,"Zürich, Rehalp","Zürich, Friedhof Enzenbühl",47.350894,8.582913,47.351029,8.580413
1,0,1800467,True,1,90,90,"Zürich, Rehalp","Zürich, Friedhof Enzenbühl",47.350894,8.582913,47.351029,8.580413
2,0,1900466,False,3,90,87,"Zürich, Balgrist","Zürich, Burgwies",47.354633,8.574880,47.357850,8.571908
3,0,1900466,True,2,90,88,"Zürich, Balgrist","Zürich, Burgwies",47.354633,8.574880,47.357850,8.571908
4,0,1900467,False,3,89,87,"Zürich, Balgrist","Zürich, Friedhof Enzenbühl",47.354246,8.575333,47.351212,8.579865


### Covert to GeoJson

We store the coordinates as a GeoJson file as this is a standard format and allows easy visualisation. The geojson defines how the lines should look like (including line color, style, etc.) -> This will be drawn on the map later on.

First, we introduce a helper function that draws triangles that will be used as arrowheads of vectors

In [23]:
import math

def arrowhead_coordinates(x1, y1, x2, y2, length=10):
    # Calculate the direction from (x1, y1) to (x2, y2)
    dx = x2 - x1
    dy = y2 - y1
    
    # Calculate the angle of the vector (x2 - x1, y2 - y1) with respect to the positive x-axis
    angle = math.atan2(dy, dx)
    
    # Calculate the coordinates of the two points forming the arrowhead
    point1_x = x2 - length * math.cos(angle - math.pi / 6)  # Subtract pi/6 to adjust for arrowhead angle
    point1_y = y2 - length * math.sin(angle - math.pi / 6)
    
    point2_x = x2 - length * math.cos(angle + math.pi / 6)  # Add pi/6 to adjust for arrowhead angle
    point2_y = y2 - length * math.sin(angle + math.pi / 6)
    
    # Return the coordinates of the arrowhead triangle
    return [(x2, y2), (point1_x, point1_y), (point2_x, point2_y)]

# Example usage:
x1, y1 = 0, 0
x2, y2 = 10, 0
arrow_coords = arrowhead_coordinates(x1, y1, x2, y2, length=1)
print("Arrowhead coordinates:", arrow_coords)

Arrowhead coordinates: [(10, 0), (9.13397459621556, 0.49999999999999994), (9.13397459621556, -0.49999999999999994)]


Next, we create and store the Geojson file itself.

In [24]:
# Later, we will draw each line as a color. Here, we define a mapping from seat occupancy to a color
# Inserting the color already here in the JSON allows better visualization later on...
norm = mpl.colors.Normalize(vmin=-100, vmax=100) # we use -100 since we don't want to use the entire spectrum of the colormap
cmap = cm.turbo
color_mapper = cm.ScalarMappable(norm=norm, cmap=cmap)


#define function to manually output geojson file (slimmer than geopandas creates its files!)
proctable=connections_coord.copy()
def df_to_geojson(proctable, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in proctable.iterrows():

        # Draw the line
        feature = {'type':'Feature',
                   'properties':{'stroke': mcolors.to_hex(color_mapper.to_rgba(row['seat_occupancy_mean']))},
                   'geometry':{'type':'LineString',
                               'coordinates':[[]]}}
        feature['geometry']['coordinates'] = [[row['long_from'],row['lat_from']], [row['long_to'],row['lat_to']]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)

        # Draw the arrow head
        arrow_coords = arrowhead_coordinates(row['long_from'],row['lat_from'], row['long_to'],row['lat_to'], length=0.0001)
        feature = {'type':'Feature',
                   'properties':{'stroke': mcolors.to_hex(color_mapper.to_rgba(row['seat_occupancy_mean']))},
                   'geometry':{'type':'Polygon',
                               'coordinates':[[]]}}
        feature['geometry']['coordinates'] = [[[arrow_coords[0][0], arrow_coords[0][1]], [arrow_coords[1][0], arrow_coords[1][1]], [arrow_coords[2][0], arrow_coords[2][1]]]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)

        #print(geojson['features'])
        #break

    
    return geojson

#define attributes to be included in geojson file
col = ['FZ_AB', 'weekday', 'seat_occupancy_mean', 'free_seats_max', 'free_seats_mean', 'name_from', 'name_to']

#output
geojson = df_to_geojson(proctable, col)
output_filename = "vbz.geojson"
with open(output_filename, 'w', encoding='utf-8') as output_file:
    json.dump(geojson, output_file, separators=(", ", ": "), ensure_ascii=False)

# Visualize the Data

In [25]:
vbz_geo = gpd.read_file('vbz.geojson')

In [26]:
vbz_geo.head()

,stroke,FZ_AB,weekday,seat_occupancy_mean,free_seats_max,free_seats_mean,name_from,name_to,geometry
0,#a7fc3a,0,False,1,90,89,"Zürich, Rehalp","Zürich, Friedhof Enzenbühl","LINESTRING (8.58291 47.35089, 8.58041 47.35103)"
1,#a7fc3a,0,False,1,90,89,"Zürich, Rehalp","Zürich, Friedhof Enzenbühl","POLYGON ((8.58041 47.35103, 8.58050 47.35097, ..."
2,#a7fc3a,0,True,1,90,90,"Zürich, Rehalp","Zürich, Friedhof Enzenbühl","LINESTRING (8.58291 47.35089, 8.58041 47.35103)"
3,#a7fc3a,0,True,1,90,90,"Zürich, Rehalp","Zürich, Friedhof Enzenbühl","POLYGON ((8.58041 47.35103, 8.58050 47.35097, ..."
4,#acfb38,0,False,3,90,87,"Zürich, Balgrist","Zürich, Burgwies","LINESTRING (8.57488 47.35463, 8.57191 47.35785)"


First, we define a helper function that draws a title on each visualization

In [27]:
def add_title(map_, time, weekday):
    if int(time) < 12:
        time_str = f"{time}am-{int(time)+1}am"
    elif int(time) == 12:
        time_str = f"{12}am-{1}pm"
    else:
        time_str = f"{int(time)-12}pm-{int(time)-11}pm"
    daytype_str = "Weekday" if weekday else "Weekend"
    title_html = f'<h1 style="position:absolute;z-index:100000;left:20px;color:#bbb; bottom:20px" >Time: {time_str} ({daytype_str})</h1>'
    map_.get_root().html.add_child(folium.Element(title_html))
    return map_

We create a map using `folium` and directly draw on it the data using the previously generated json.

In [28]:
locations = {'place': ['Zurich University', 'Irchel Campus', 'Main Station'], 'latitude': [47.376009572041205, 47.398244842674096, 47.37801581583341], 'longitude': [8.548333, 8.549042321653372, 8.540456430522084]}
locations = pd.DataFrame(locations)

def get_map(time, weekday):
    zurich_map = folium.Map(location=[47.38, 8.55], tiles="Cartodb dark_matter", zoom_start=13)
  
    def plot_locations(point):
        folium.Marker(location=[point.latitude, point.longitude],
                      tooltip=point.place, name="Locations").add_to(zurich_map)

    locations.apply(plot_locations, axis=1)
    
    vbz_geo_show = vbz_geo[(vbz_geo.FZ_AB == int(time)) & (vbz_geo.weekday == weekday)]

    folium.GeoJson(
      vbz_geo_show.to_json(),
      name="VBZ Data",
      control=True,
      style_function = lambda x: {'color': x['properties']['stroke'], 'fillColor': x['properties']['stroke'],  'opacity': 0.9, 'weight': 3.0, 'fillOpacity': 1.0},
      highlight_function = lambda x: {'opacity': 1.0, 'weight': 4.0, 'fillOpacity': 1.0},
      tooltip=folium.features.GeoJsonTooltip(
          fields=['name_from', 'name_to', 'seat_occupancy_mean', 'free_seats_mean'],
          aliases=['From: ', 'To: ', 'Average Seat Occupation[%]: ', 'Average Free Seats: '],
          style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") # setting style for popup box
        )
      ).add_to(zurich_map)
    
    folium.LayerControl().add_to(zurich_map)
    zurich_map = add_title(zurich_map, time, weekday)
    

    return zurich_map

time = input("Please enter a time [0-24]")
weekday = input("Weekday [yes/no]?")
weekday = (weekday.lower() == "yes")

zurich_map = get_map(time, weekday)
zurich_map

Please enter a time [0-24] 8
Weekday [yes/no]? yes


Save the map as HTML

In [29]:
# zurich_map.save(f'map_{time}_{weekday}.html')

### Create all maps and store them as HTML

Create all maps that later can be viewed in the web browser


In [31]:
maps_path = Path("results/")

if not maps_path.exists():
    maps_path.mkdir()
else:
    assert not any(maps_path.iterdir()), "Result folder should be empty...)"

count = 0
for weekday in [True, False]:
  for time in range(24):
    zurich_map = get_map(time, weekday)
    count += 1
    try:
      zurich_map.save(f'{str(maps_path)}/map_{count:02d}_{time}_{weekday}.html')
    except Exception:
      print(f"No data for {time} {weekday}")

### Create Video

We also convert the HTML to images to create a video that shows the development of traffic over time.

In [32]:
hti = Html2Image(custom_flags=['--virtual-time-budget=10000', '--hide-scrollbars'], output_path=str(maps_path))

for file in maps_path.glob("map_*.html"):
    hti.screenshot(html_file=str(file), save_as=f"{str(file.name).split('.')[0]}.png")

[0411/082508.883375:WARNING:runtime_features.cc(728)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.
[0411/082509.181907:WARNING:runtime_features.cc(728)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.
1236468 bytes written to file /Users/sage/Documents/Projects/uzh-data-science-project/src/results/map_09_8_True.png
[0411/082511.258935:WARNING:runtime_features.cc(728)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.
[0411/082511.374015:WARNING:runtime_features.cc(728)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.
1234741 bytes writte

In [33]:
video_name = 'traffic.mp4'

images = sorted(list(maps_path.glob("map_*.png")))
images = [str(img) for img in images]
frame = cv2.imread(images[0])
height, width, layers = frame.shape

fps = 1
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

for image in images:
    video.write(cv2.imread(image))

video.release()

# Visualize Boarding and Alighting Passengers

So far, we focused on the connections and how occupied the vehicles are. Next, we focus on the stops itself and visualize how many passengers are entering / leaving the vehicles.

In [34]:
data.head()

,Tagtyp_Id,Linien_Id,Linienname_x,Plan_Fahrt_Id,Richtung,Sequenz,Haltestellen_Id_from,Nach_Hst_Id,FZ_AB,Anzahl_Messungen,...,Haltestellenkurzname_to,Haltestellenlangname_to,GPS_Latitude,GPS_Longitude,Haltestellen_Id,GPS_Latitude_to,GPS_Longitude_to,weekday,seat_occupancy,free_seats
0,21,113,15,116279,1,1,161,160,9,9,...,HIRS,"Zürich, Hirschwiesenstrasse",47.397778,8.541750,129,47.400252,8.543470,False,0.006734,65.55556
1,21,113,15,116279,1,2,160,159,9,9,...,BERN,"Zürich, Berninaplatz",47.400252,8.543470,400,47.403543,8.547780,False,0.013468,65.11111
2,21,113,15,116279,1,3,159,158,9,9,...,SALE,"Zürich, Salersteig",47.403543,8.547780,399,47.406055,8.548362,False,0.072391,61.22222
3,21,113,15,116279,1,4,158,26,9,9,...,SOER,"Zürich, Sternen Oerlikon",47.406055,8.548362,398,47.410066,8.546233,False,0.089226,60.11111
4,21,113,15,116279,1,5,26,25,9,9,...,BOER,"Zürich, Bahnhof Oerlikon",47.410066,8.546233,49,47.411274,8.544966,False,0.063973,61.77778


In [35]:
# Calculate the boarding and alighting passengers per stop
passengers_stops = data.groupby(['FZ_AB', 'ID_Abschnitt', 'weekday'])
passengers_stops = passengers_stops.agg(
    boarding=('Einsteiger', 'sum'),
    alighting=('Aussteiger', 'sum'),
    name=('Haltestellenlangname_from', 'first'),
    lat=('GPS_Latitude', 'first'),
    long=('GPS_Longitude', 'first'),
)

# Reset index (otherwise, FZ_AB is the index)
passengers_stops = passengers_stops.reset_index()

passengers_stops.head()

,FZ_AB,ID_Abschnitt,weekday,boarding,alighting,name,lat,long
0,0,1800467,False,10.86473,0.00000,"Zürich, Rehalp",47.350878,8.583213
1,0,1800467,True,9.65796,0.00000,"Zürich, Rehalp",47.350878,8.583213
2,0,1900466,False,21.56248,0.67262,"Zürich, Balgrist",47.354413,8.575083
3,0,1900466,True,15.49105,0.52133,"Zürich, Balgrist",47.354413,8.575083
4,0,1900467,False,0.70865,45.86016,"Zürich, Balgrist",47.354413,8.575083


In [36]:
# Sum up the boarding and alighting passengers
passengers_stops['passengers'] = passengers_stops.boarding + passengers_stops.alighting
passengers_stops.drop(['boarding', 'alighting'], axis=1, inplace=True)
passengers_stops.head()

,FZ_AB,ID_Abschnitt,weekday,name,lat,long,passengers
0,0,1800467,False,"Zürich, Rehalp",47.350878,8.583213,10.86473
1,0,1800467,True,"Zürich, Rehalp",47.350878,8.583213,9.65796
2,0,1900466,False,"Zürich, Balgrist",47.354413,8.575083,22.23510
3,0,1900466,True,"Zürich, Balgrist",47.354413,8.575083,16.01238
4,0,1900467,False,"Zürich, Balgrist",47.354413,8.575083,46.56881


## Normalize the data

There is quite some fluctuation in passengers per stop and per hour. We normalize the data across all deaprture times / weekdays so that we can later compare the heatmaps to each other. (Otherwise, the library uses the highest value per hour as its maximum value, when remaing as weight it is between 0..1)

In [37]:
print("Max passengers per stop per hour:", passengers_stops.passengers.max())
print("Min passengers per stop per hour:", passengers_stops.passengers.min())

Max passengers per stop per hour: 8460.93263
Min passengers per stop per hour: 0.0


In [38]:
passengers_stops.passengers /= passengers_stops.passengers.max()
passengers_stops.rename(columns={"passengers": "weight"}, inplace=True)

### Create the Heatmap

In [39]:
def get_map_passengers(time, weekday):
    zurich_map = folium.Map(location=[47.38, 8.55], tiles="Cartodb dark_matter", zoom_start=13)

    passengers_stops_vis = passengers_stops[(passengers_stops.FZ_AB == int(time)) & (passengers_stops.weekday == weekday)]
    passengers_stops_vis.drop(['FZ_AB', 'ID_Abschnitt', 'weekday', 'name'], axis=1, inplace=True)
    
    HeatMap(passengers_stops_vis, 
        min_opacity=0.4,
        blur = 10
               ).add_to(folium.FeatureGroup(name='Heat Map').add_to(zurich_map))
    folium.LayerControl().add_to(zurich_map)
    
    zurich_map = add_title(zurich_map, time, weekday)

    return zurich_map

heat_map = get_map_passengers(8, True)
heat_map

Store all the maps

In [40]:
count = 0
for weekday in [True, False]:
  for time in range(24):
    zurich_map = get_map_passengers(time, weekday)
    count += 1
    try:
      zurich_map.save(f'{str(maps_path)}/heatmap_{count:02d}_{time}_{weekday}.html')
    except Exception:
      print(f"No data for {time} {weekday}")

### Create a video

First, we convert HTML to PNG, next we build an mp4 based on the images

In [41]:
for file in maps_path.glob("heatmap_*.html"):
    hti.screenshot(html_file=str(file), save_as=f"{str(file.name).split('.')[0]}.png")

[0411/082649.606990:WARNING:runtime_features.cc(728)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.
[0411/082649.707447:WARNING:runtime_features.cc(728)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.
1551865 bytes written to file /Users/sage/Documents/Projects/uzh-data-science-project/src/results/heatmap_16_15_True.png
[0411/082651.116376:WARNING:runtime_features.cc(728)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.
[0411/082651.212087:WARNING:runtime_features.cc(728)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.
1552421 bytes w

In [42]:
video_name = 'stops.mp4'

images = sorted(list(maps_path.glob("heatmap_*.png")))
images = [str(img) for img in images]
frame = cv2.imread(images[0])
height, width, layers = frame.shape

fps = 1
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

for image in images:
    video.write(cv2.imread(image))

video.release()